In [9]:
import pandas as pd
import pathlib
from pathlib import Path


from ucimlrepo import fetch_ucirepo

# Assume notebook lives in repo_root/notebooks/
repo_root = pathlib.Path().resolve().parent
raw_data_dir = repo_root / "data" / "raw"
raw_data_dir.mkdir(parents=True, exist_ok=True)

print("Saving to:", raw_data_dir)

# Fetch dataset
heart_disease = fetch_ucirepo(id=45)
X = heart_disease.data.features
y = heart_disease.data.targets

# Combine feature + target for easier analysis
df = pd.concat([X, y], axis=1)

# Save to the correct folder (regardless of where your notebook lives)
df.to_csv(raw_data_dir / "heart_disease.csv", index=False)

print("Saved:", raw_data_dir / "heart_disease.csv")

Saving to: C:\Repos\ml-classification-karlidean\data\raw
Saved: C:\Repos\ml-classification-karlidean\data\raw\heart_disease.csv
